In [1]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
from bs4 import BeautifulSoup
import json
import time
import os
import pickle
import pprint
from pyvirtualdisplay import Display #use linuxbrew to install chromedriver if you want to do this on AWS instead of your local machine

First, we'll use the Portland Maps API to get all homes sold in Portland over a two year span. https://www.portlandmaps.com/development/

In [11]:
# https://github.com/psf/requests/issues/3829
# alternate not tried: https://stackoverflow.com/questions/48391750/disable-python-requests-ssl-validation-for-an-imported-module 
# probably better: https://stackoverflow.com/questions/39356413/how-to-add-a-custom-ca-root-certificate-to-the-ca-store-used-by-pip-in-windows/52961564#52961564 
# https://curl.haxx.se/docs/sslcerts.html
class WrappedSession(requests.Session):
    """A wrapper for requests.Session to override 'verify' property, ignoring REQUESTS_CA_BUNDLE environment variable.

    This is a workaround for https://github.com/kennethreitz/requests/issues/3829 (will be fixed in requests 3.0.0)
    """
    def merge_environment_settings(self, url, proxies, stream, verify, *args, **kwargs):
        if self.verify is False:
            verify = False

        return super(WrappedSession, self).merge_environment_settings(url, proxies, stream, verify, *args, **kwargs)
    


WrappedSession(requests.get('https://www.google.com',verify=False))

In [13]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [3]:
requests.get('https://www.google.com')

<Response [200]>

In [20]:
#get all houses sold since between 7/1/16 and 6/30/17 from Portland Maps
addresses = []


kwargs = {'api_key': '###',
    'sqft_larger': '1',
    'sold_after': '07/01/2016',
    'sold_before': '06/30/2017',
    'page':'1',
    'format': 'json'
    }
    
u = 'http://www.portlandmaps.com/api/assessor'
request = requests.get(u, params=kwargs)
js = json.loads(request.text)

print(js['total'])
    
    
for i in range(len(js['results'])):
    if (js['results'][i]['address']):
        addresses.append(js['results'][i])
            
if int(js['total']) > 1000:
    pages = int(js['total'])//1000
    
    for i in range(2,pages+2):
        kwargs = {'api_key': '###',
        'sqft_larger': '1',
        'sold_after': '07/01/2016',
        'sold_before': '06/30/2017',
        'page': i,
        'format': 'json'
    }
    
        u = 'https://www.portlandmaps.com/api/assessor'
        request = requests.get(u, params=kwargs)
        js = json.loads(request.text)
        
        for i in range(len(js['results'])):
            if (js['results'][i]['address']):
                addresses.append(js['results'][i])
    

35009


In [21]:
addresses = pd.DataFrame(addresses)
addresses = addresses[['address', 'city', 'latitude', 'longitude', 'legal_description', 'market_value', 'neighborhood', 'owner', 'property_id', 'sale_date', 'sale_price', 'square_feet', 'year_built', 'zip_code']]
addresses = addresses[addresses['city'] == 'PORTLAND']
addresses = addresses[addresses['neighborhood'] != 'HAYDEN ISLAND NEIGHBORHOOD NETWORK'] #get rid of floating homes
addresses.reset_index(drop=True, inplace=True)
print(len(addresses))
addresses.head()

12965


,address,city,latitude,longitude,legal_description,market_value,neighborhood,owner,property_id,sale_date,sale_price,square_feet,year_built,zip_code
0,01100 SW PALATINE HILL RD,PORTLAND,45.44406,-122.66709,"PALATINE HILL, LOT 45 TL 1000",1299050.0,MULT CO RIVERDALE AREA,"VENTURA,KATHLEEN L & O'NEILL,JAMES",R232835,2017-06-07,1250000,3952,1943,97219.0
1,01115 SW COMUS ST,PORTLAND,45.44701,-122.66653,"PALATINE HILL 3, BLOCK 106, LOT 15&16",657150.0,MULT CO RIVERDALE AREA,"CARLSON,JORDAN & CARLSON,ALISHA",R232939,2016-09-13,650000,1533,1968,97219.0
2,01250 SW RADCLIFFE RD,PORTLAND,45.45065,-122.66573,"SECTION 27 1S 1E, TL 2500 0.36 ACRES",837050.0,MULT CO RIVERDALE AREA,"BEST,DUSTIN & BEST,KRISTEN J",R330649,2016-08-24,880000,2832,1967,97219.0
3,01510 SW BUDDINGTON ST,PORTLAND,45.44247,-122.66318,"SECTION 34 1S 1E, TL 1400 1.00 ACRES",2914590.0,MULT CO RIVERDALE AREA,"COOKS,BRANDIN",R331686,2016-12-13,2850000,8189,2012,97219.0
4,01551 SW MILITARY RD,PORTLAND,45.44107,-122.66245,"ABERNETHY HTS, LOT 42 TL 500",1211680.0,MULT CO RIVERDALE AREA,KARL SCHULZ MANAGEMENT GROUP LLC,R100303,2017-06-20,880000,3275,1985,97219.0


Now, we'll get the Portland Maps API details on each of the homes

In [ ]:
#get full Portlandmaps detail of each of the above properties

count = 1
for i in range(len(addresses)):
    kwargs = {'api_key': '###',
    'detail_type': 'assessor',
    'sections': '*',
    'detail_id': addresses['property_id'][i],
    'format': 'json'
    }

    u = 'https://www.portlandmaps.com/api/detail'
    request = requests.get(u, params=kwargs)
    if request.status_code == 429:
        break
    
    if count % 150 == 0:
        count = 1
        time.sleep(600)
    else:    
        count = count + 1
        time.sleep(2)
    
    if count % 10 == 0:
        addresses.to_pickle('home_results.pkl')
        
    print(i,request)
    js = json.loads(request.text)
    try:
        addresses.set_value(i, 'prop_tax_yr_of_sale', js['tax ' 'history'][1]['property_tax'])
        addresses.set_value(i, 'lot_size', js['general']['total_land_area_sqft'])
        addresses.set_value(i, 'type', js['improvements']['building_class'])
        addresses.set_value(i, 'plumbing', js['improvements']['plumbing'])
        addresses.set_value(i, 'description',js['improvements']['improvement_type'])
        addresses.set_value(i, 'main_sqft', js['improvements']['details'][0]['area_sq_ft'])
        addresses.set_value(i, 'segments', js['improvements']['number_of_segments'])
    except:
        continue

0 <Response [200]>
1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>
10 <Response [200]>
11 <Response [200]>
12 <Response [200]>
13 <Response [200]>
14 <Response [200]>
15 <Response [200]>
16 <Response [200]>
17 <Response [200]>
18 <Response [200]>
19 <Response [200]>
20 <Response [200]>
21 <Response [200]>


Now, we'll use the Zillow API to get more details on each of the homes. You need to make two API calls (ugh!) -  one to get the Zillow ID, and one to get the home details. 

In [ ]:
#zillow keys
#get a lot of keys because zillow has a 1000 call per day per key limit

keys1 = ['###', '###', '###']

keys2 = ['###', '###', '###']

In [ ]:
count = 0
for i in range(len(addresses)):
    
    if i & 1000 == 0: #API limits
        count = count + 1
        key = keys1[count]
        
    print(i, key)

    kwargs = {'zws-id': key,
        'address': addresses['address'][i],
        'citystatezip': str(addresses['city'][i]) + ' OR ' + str(addresses['zip_code'][i])
        }

    u = 'http://www.zillow.com/webservice/GetDeepSearchResults.htm'
    request = requests.get(u, params=kwargs)

    time.sleep(2)

    try:
        
        soup = BeautifulSoup(request.text,'html.parser')
  
        addresses.set_value(i, 'zillow_id',  soup.find_all('zpid')[0].text)
        addresses.set_value(i, 'zillow_finished_sqft', soup.find_all('finishedsqft')[0].text)
        addresses.set_value(i,'zillow_bedrooms', soup.find_all('bedrooms')[0].text)

    except:
        continue    

In [ ]:
#use this Zillow API to get school and realtor info

count = 0
for i in range(len(addresses)):
    
    if i & 1000 == 0: #API limits
        count = count + 1
        key = keys2[count]
        
    print(count, key)
    
    try:
        zillow_id = int(addresses['zillow_id'][i])
        count = count + 1
        
        kwargs = {'zws-id': key,
        'zpid': zillow_id
        }

        u = 'http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm'
        request = requests.get(u, params=kwargs)       
        time.sleep(2)       
        print(i,request, count)
        soup = BeautifulSoup(request.text,'html.parser')
        request = request.text
    except:
        continue


    try:
        addresses.set_value(i, 'numrooms',  soup.find_all('numrooms')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'roof',  soup.find_all('roof')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'exteriormaterial',  soup.find_all('exteriormaterial')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'parkingtype',  soup.find_all('parkingtype')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'coveredparkingspaces',  soup.find_all('coveredparkingspaces')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'heatingsources',  soup.find_all('heatingsources')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'heatingsystem',  soup.find_all('heatingsystem')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'coolingsystem',  soup.find_all('coolingsystem')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'appliances',  soup.find_all('appliances')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'floorcovering',  soup.find_all('floorcovering')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'architecture',  soup.find_all('architecture')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'homedescription',  soup.find_all('homedescription')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'elementaryschool',  soup.find_all('elementaryschool')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'middleschool',  soup.find_all('middleschool')[0].text)
    except:
        pass
    try:
        addresses.set_value(i, 'highschool',  soup.find_all('highschool')[0].text)
    except:
        pass


In [ ]:
addresses.to_pickle('zillow.pkl')

Now, we'll use need to get Redfin URLS of the houses in order to later download photos.

In [ ]:
for i in range(len(addresses)):
    if i % 100 == 0: #reload your browser once in a while
        chromedriver = "/usr/local/bin/chromedriver"
        driver = webdriver.Chrome(chromedriver)
#        display = Display(visible=0, size=(1300, 1080)) #use these lines for AWS instead of local
#        display.start()
#        chromedriver = "/home/ubuntu/.linuxbrew/bin/chromedriver"
#        driver = webdriver.Chrome(chromedriver)
#        driver.set_window_size(1300, 1080)
    try:
        url = 'https://images.google.com/'
#        driver.get("https://images.google.com/") #use this for AWS instead of local
        driver.get(url)

        search = driver.find_element_by_xpath('//*[@id="lst-ib"]')

        address = addresses['address'][i] + ' Portland OR Redfin'

        search.send_keys(address)

        search.send_keys(Keys.RETURN)  # XXX: problem here.

        pic = driver.find_element_by_xpath("//*[@id='rg_s']/div[1]/a/img")

        pic.click()
        time.sleep(5)
    
        download_pic = driver.find_element_by_xpath("//*[@id='irc_cc']/div[2]/div[1]/div[2]/div[2]/a/img")
        time.sleep(2.5)
    
        src = download_pic.get_attribute('src')
        print(i, src)
    
        addresses.set_value(i, 'url', src)
    except:
        continue

    if (i-1)%100 == 0:
        driver.close()


In [ ]:
addresses.to_pickle('zillow.pkl')

Now, get the photos. You need to hide your user agent from Redfin because they don't let you scrape directly


In [ ]:
#you must hide user agent from redfin!
for i in range(0,len(addresses)):
    link = addresses['url'][i]
    print(link)
    filename = 'pics/'+str(i)+'.png'
    if 'https://ssl.cdn-redfin.com' in addresses['url'][i]:
        !curl {link} -s -H 'Pragma: no-cache' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-US,en;q=0.8' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36' -H 'Content-Type:application/x-www-form-urlencoded' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'Cache-Control: no-cache'  -H 'Connection: keep-alive' --compressed > {filename}
    else:
        !curl {link} > {filename}
    time.sleep(2)

In [ ]:
#if any files got messed up, try again:

#for files that don't exist:
for i in range(0,len(addresses)):
    link = addresses['url'][i]
    print(link)
    filename = 'pics/'+str(i)+'.png'
    if not os.path.exists(filename):
        if 'https://ssl.cdn-redfin.com' in link:
            !curl {link} -s -H 'Pragma: no-cache' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-US,en;q=0.8' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36' -H 'Content-Type:application/x-www-form-urlencoded' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'Cache-Control: no-cache'  -H 'Connection: keep-alive' --compressed > {filename}
        else:
            urllib.request.urlretrieve(link, filename)
    time.sleep(2)

#for files that didnt download correctly:
for i in range(0,len(addresses)):
    link = addresses['url'][i]
    print(link)
    filename = 'pics/'+str(i)+'.png'
    if os.path.getsize(filename) < 500 :
        if 'https://ssl.cdn-redfin.com' in link:
            !curl {link} -s -H 'Pragma: no-cache' -H 'Accept-Encoding: gzip, deflate, br' -H 'Accept-Language: en-US,en;q=0.8' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36' -H 'Content-Type:application/x-www-form-urlencoded' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'Cache-Control: no-cache'  -H 'Connection: keep-alive' --compressed > {filename}
        else:
            urllib.request.urlretrieve(link, filename)
    time.sleep(2)


Are any files still messed up? If so, delete those entries.

In [ ]:
indices = []
for i in range(len(addresses)):
    filename = 'pics/'+str(i)+'.png'
    if os.path.getsize(filename) < 500 or if not os.path.exists(filename):
        indices.append(i)
addresses = addresses.drop(addresses.index[indices])


In [ ]:
addresses.to_pickle('year2/home_results.pkl')
addresses.to_csv('year2/home_results.csv')